In [0]:
import collections
import scipy.linalg
import scipy.integrate
import numpy as np

PendulumParams = collections.namedtuple(
  "PendulumParams", "length mass g"
)

In [27]:
def pendulum_dynamics(params):
  
  def dynamics(x, u, t):
    del u, t
    w = np.product(params)*np.sin(x[0])
    # assume point mass and massless arm
    inertia = params.mass * params.length**2
    amat = np.array([[0, 1], [w/inertia, 0]])
    bmat = np.array([[0], [np.reciprocal(inertia)]])
    return amat, bmat
  
  return dynamics


def dynamical_system(dynamics):
  
  def compute_dynamics(x, u, t):
    amat, bmat = dynamics(x, u, t)
    return np.dot(amat, x) + np.dot(bmat, u)
  
  return compute_dynamics


def controlled_system(policy, system):
  
  def compute_dxdt(x, t):
    u = policy(x, t)
    dxdt = system(x, u, t)
    return dxdt
  
  return compute_dxdt


def continuous_lqr_solve(dynamics, x_goal, t, qmat, rmat):
  amat, bmat = dynamics(x_goal, np.zeros(1), t)
  pmat = scipy.linalg.solve_continuous_are(amat, bmat, qmat, rmat)
  return scipy.linalg.solve(rmat, np.dot(bmat.T, pmat))
  

def lqr_policy(kmat, x_goal):
  def policy(x, t):
    return np.dot(kmat, x_goal - x)
  return policy
                  

x_init = np.array([np.pi/2 - 0.3, 0.])
x_goal = np.array([np.pi/2, 0.])
params = PendulumParams(mass=1., length=1., g=-9.81)


dynamics = pendulum_dynamics(params)
kmat = continuous_lqr_solve(dynamics, x_goal, 0.,
                            np.eye(2), np.ones(1)*0.00001)
policy = lqr_policy(kmat, x_goal)

sp.integrate.odeint(controlled_system(policy, dynamical_system(dynamics)),
                    x_init, np.linspace(0, 10., 300))

array([[1.27079633e+00, 0.00000000e+00],
       [1.27834668e+00, 2.45571736e-01],
       [1.28642273e+00, 2.37416005e-01],
       [1.29423040e+00, 2.29526355e-01],
       [1.30177861e+00, 2.21900141e-01],
       [1.30907605e+00, 2.14528518e-01],
       [1.31613108e+00, 2.07402934e-01],
       [1.32295181e+00, 2.00515139e-01],
       [1.32954603e+00, 1.93857159e-01],
       [1.33592132e+00, 1.87421284e-01],
       [1.34208497e+00, 1.81200060e-01],
       [1.34804404e+00, 1.75186285e-01],
       [1.35380535e+00, 1.69373002e-01],
       [1.35937550e+00, 1.63753485e-01],
       [1.36476086e+00, 1.58321235e-01],
       [1.36996758e+00, 1.53069970e-01],
       [1.37500161e+00, 1.47993630e-01],
       [1.37986871e+00, 1.43086350e-01],
       [1.38457443e+00, 1.38342467e-01],
       [1.38912413e+00, 1.33756507e-01],
       [1.39352304e+00, 1.29323173e-01],
       [1.39777614e+00, 1.25037358e-01],
       [1.40188832e+00, 1.20894116e-01],
       [1.40586424e+00, 1.16888679e-01],
       [1.409708